In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_csv('CCES2016.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])
df2 = pd.read_csv('CCES2.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])
df3 = pd.read_csv('CCES3.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])

#'utf-8' codec can't decode byte 0xe1 in position 5: unexpected end of data

df = df1.append(df2)
df = df.append(df3)

df = df.reset_index()

In [ ]:
df.head()

The birth year ("birthyr") column is missing in the csv files containing the 2016 CCES data. Fortunately, I was able to find them in another spreadsheet that showed the cumulative CCES results from 2006 to 2017, which uses the same case ID index. The cumuative16.csv file contains the rows from that spreadsheet for the 2016 survey.

In [4]:
cumulative = pd.read_csv('cumulative16.csv', encoding = 'latin1')
cumulative = cumulative[['case_id', 'birthyr']]
cumulative.head()

In [5]:
df = pd.merge(df, cumulative, left_on = 'V101', right_on = 'case_id')

In [6]:
cumulative.head()

,case_id,birthyr
0,222168628,1969
1,273691199,1994
2,284214415,1964
3,287557695,1988
4,290387662,1982


In [11]:
df['birthyr'] = df['birthyr_y']

In [15]:
df.drop(['birthyr_y'], axis = 1)
df[['V101', 'birthyr']].head()

,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,tookpost,lookupzip,inputstate,cdid113,...,CL_E2016PEP,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname,case_id,birthyr
0,222168628,1.344214,1.042267,1.195346,0.944967,0.730450,Yes,3442,New Hampshire,2,...,NaN,NaN,05:44.9,36:38.8,24:32.9,40:18.8,NaN,NaN,222168628,1969
1,273691199,1.182352,1.280103,1.280565,0.767256,0.892838,Yes,70118,Louisiana,2,...,NaN,NaN,54:58.5,20:41.7,02:55.1,09:26.9,622.0,NEW ORLEANS,273691199,1994
2,284214415,0.217396,NaN,0.248930,0.090604,0.000000,No,63851,Missouri,8,...,NaN,NaN,54:12.8,24:44.8,NaN,NaN,632.0,PADUCAH-CAPE GIRARD-HARSBG,284214415,1964
3,287557695,0.532320,NaN,0.928750,0.142583,0.000000,No,36703,Alabama,7,...,NaN,NaN,06:21.3,26:00.1,NaN,NaN,698.0,MONTGOMERY-SELMA,287557695,1988
4,290387662,1.261106,1.092519,1.161894,1.206498,1.019007,Yes,80526,Colorado,2,...,NaN,NaN,56:31.1,14:12.2,09:28.0,21:05.9,751.0,DENVER,290387662,1982
5,290932100,0.530663,0.542908,0.519627,0.887728,0.872255,Yes,35816,Alabama,5,...,NaN,NaN,15:32.1,40:58.0,45:30.2,56:20.9,691.0,HUNTSVILLE-DECATUR (FLOR),290932100,1963
6,292860642,1.446300,1.382383,1.373394,1.112030,1.009516,Yes,75402,Texas,4,...,NaN,NaN,18:03.0,45:30.9,01:34.7,15:51.9,623.0,DALLAS-FT. WORTH,292860642,1962
7,295367942,1.694283,NaN,NaN,2.287548,0.000000,No,16833,Pennsylvania,5,...,NaN,NaN,59:43.8,55:36.4,NaN,NaN,574.0,JOHNSTOWN-ALTOONA-ST COLGE,295367942,1991
8,295717127,2.400476,2.003689,2.092420,2.414978,1.887757,Yes,30711,Georgia,14,...,REP,polling,44:48.5,08:47.6,17:49.2,27:48.1,575.0,CHATTANOOGA,295717127,1963
9,295859014,0.888128,0.779780,0.828414,0.797421,0.682409,Yes,17522,Pennsylvania,16,...,NaN,NaN,36:17.8,05:00.5,25:18.7,16:59.5,566.0,HARRISBURG-LNCSTR-LEB-YORK,295859014,1957


In [14]:
#ensure that case IDs and birth years match for both dataframes
different_index = []
for i in range(64600):
    if df['V101'].iloc[i] != cumulative['case_id'].iloc[i]:
        different_index.append(i)
        
different_year = []
for i in range(64600):
    if df['birthyr'].iloc[i] != cumulative['birthyr'].iloc[i]:
        different_year.append((i, df['birthyr'].iloc[i], cumulative['birthyr'].iloc[i] ))
        
print(len(different_index))
print(len(different_year))

0
0


In [16]:
#print list of columns in dataframe

columns_indices = []

for i in range(len(df.columns)):
    #if "ed" in df.columns[i]:
    columns_indices.append((i, df.columns[i]))
    
columns_indices = sorted(columns_indices, key = lambda x: x[1].lower())

for item in columns_indices:
    if item[0] < 10:
        print(str(item[0]) + "   " + str(item[1]))
    elif item[0] < 100:
        print(str(item[0]) + "  " + str(item[1]))
    else:
        print(str(item[0]) + " " + str(item[1]))

70  add_confirm
104 Asian_origin_1
113 Asian_origin_10
114 Asian_origin_11
115 Asian_origin_12
116 Asian_origin_13
117 Asian_origin_14
118 Asian_origin_15
105 Asian_origin_2
106 Asian_origin_3
107 Asian_origin_4
108 Asian_origin_5
109 Asian_origin_6
110 Asian_origin_7
111 Asian_origin_8
112 Asian_origin_9
119 Asian_origin_t
372 AskParty_post
565 birthyr
75  birthyr_x
564 birthyr_y
563 case_id
121 CC16_300_1
122 CC16_300_2
123 CC16_300_3
124 CC16_300_4
125 CC16_300_5
126 CC16_300_6
127 CC16_300b
128 CC16_300c
129 CC16_300d_1
130 CC16_300d_2
131 CC16_300d_3
132 CC16_300d_4
133 CC16_300d_5
134 CC16_301a
135 CC16_301b
136 CC16_301c
137 CC16_301d
138 CC16_301e
139 CC16_301f
140 CC16_301g
141 CC16_301h
142 CC16_301i
143 CC16_301j
144 CC16_301k
145 CC16_301l
146 CC16_301m
147 CC16_301n
148 CC16_301o
149 CC16_302
150 CC16_303
151 CC16_304
152 CC16_305_1
161 CC16_305_10
162 CC16_305_11
153 CC16_305_2
154 CC16_305_3
155 CC16_305_4
156 CC16_305_5
157 CC16_305_6
158 CC16_305_7
159 CC16_305_8
160 C

In [17]:
#print number of NaN values by column

nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1])

for i in range(1, len(nan_counts)):
    print(nan_counts[i][0], nan_counts[i][1])

commonweight_vv 0
commonweight 0
commonweight_post 0
tookpost 0
lookupzip 0
inputstate 0
cdid113 0
cdid115 0
GovCand1Incumbent 0
GovCand2Incumbent 0
HouseCand1Incumbent 0
HouseCand2Incumbent 0
SenCand1Incumbent 0
SenCand2Incumbent 0
CCEStake 0
comptype 0
votereg 0
gender 0
educ 0
race 0
multrace_1 0
multrace_2 0
multrace_3 0
multrace_4 0
multrace_5 0
multrace_8 0
multrace_98 0
ccesmodule 0
CC16_300_1 0
CC16_300_2 0
CC16_300_3 0
CC16_300_4 0
CC16_300_5 0
CC16_300_6 0
CC16_305_1 0
CC16_305_2 0
CC16_305_3 0
CC16_305_4 0
CC16_305_5 0
CC16_305_6 0
CC16_305_7 0
CC16_305_8 0
CC16_305_9 0
CC16_305_10 0
CC16_305_11 0
CC16_331_1 0
CC16_331_2 0
CC16_331_3 0
CC16_331_7 0
CC16_331_9 0
employ 0
milstat_1 0
milstat_2 0
milstat_3 0
milstat_4 0
milstat_5 0
healthins_1 0
healthins_2 0
healthins_3 0
healthins_4 0
healthins_5 0
healthins_6 0
CL_matched 0
starttime_pre 0
endtime_pre 0
case_id 0
birthyr_y 0
birthyr 0
CC16_351K 19
pid3 20
faminc 21
CC16_320a 22
CC16_320c 31
CC16_334a 32
CC16_320b 33
CC16_334

HouseCand8Party_post 64435
race_other 64444
religpew_lutheran_t 64446
religpew_pentecost_t 64446
CC16_410b_nv_t 64446
HouseCand9Name 64477
HouseCand9Party 64477
HouseCand10Name 64477
HouseCand10Party 64477
HouseCand11Name 64477
HouseCand11Party 64477
religpew_catholic_t 64491
HouseCand9Name_post 64506
HouseCand9Party_post 64506
HouseCand10Name_post 64506
HouseCand10Party_post 64506
HouseCand11Name_post 64506
HouseCand11Party_post 64506
CC16_411_t 64507
religpew_presby_t 64526
religpew_buddhist_t 64529
CC16_366_t 64530
religpew_holiness_t 64535
religpew_methodist_t 64553
CC16_364b_t 64556
religpew_hindu_t 64561
religpew_jewish_t 64566
CC16_367x_t 64571
CC16_423c_other 64573
religpew_muslim_t 64574
CC16_365x_t 64579
CC16_411_nv_t 64581
religpew_episcop_t 64582
religpew_reformed_t 64583
religpew_orthodox_t 64586
religpew_christian_t 64589
religpew_congreg_t 64589
CC16_366x_t 64598
religpew_advent_t 64599
religpew_mormon_t 64599
regzip 64600
inputzip 64600
birthyr_x 64600
child18num 64600


215 variables have fewer than 10% of the data missing.

In [18]:
df.groupby(['tookpost']).count()

,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,lookupzip,inputstate,cdid113,cdid115,...,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname,case_id,birthyr_y,birthyr
tookpost,,,,,,,,,,,,,,,,,,,,,
No,11701,11701,43,4280,11701,11701,11701,11701,11701,11701,...,576,11701,11701,0,0,10827,10827,11701,11701,11701
Yes,52899,52899,52899,52868,52899,52899,52899,52899,52899,52899,...,7742,52899,52899,52899,52899,48706,48706,52899,52899,52899


52899 took the post survey.

Because the number of variables was so large, I created a spreadsheet to keep track of notes for each variable and decide whether it should be included in this project. Here I import and extract data from this spreadsheet.

In [19]:
vars_df = pd.read_csv('variable_list.csv', encoding = 'latin1')
variables = vars_df.loc[vars_df['Include'] == 'yes']['Name'].tolist()

In [20]:
print(variables)

['tookpost', 'inputstate', 'birthyr', 'gender', 'educ', 'race', 'CC16_331_1', 'CC16_331_2', 'CC16_331_3', 'CC16_331_7', 'CC16_351K', 'faminc', 'CC16_334a', 'CC16_334d', 'CC16_334c', 'CC16_333d', 'CC16_333b', 'CC16_333a', 'CC16_332a', 'CC16_332d', 'CC16_332e', 'CC16_333c', 'ownhome', 'CC16_332c', 'CC16_332f', 'CC16_351I', 'CC16_332b', 'child18', 'CC16_351B', 'CC16_351E', 'union', 'CC16_327', 'immstat', 'CC16_351G', 'CC16_351F', 'CC16_330a', 'CC16_351H', 'CC16_330e', 'CC16_330b', 'unionhh', 'CC16_330d', 'CC16_335', 'hispanic', 'sexuality', 'investor', 'trans', 'inputstate_post', 'votereg_post', 'CC16_414_1', 'CC16_414_2', 'CC16_414_3', 'CC16_414_4', 'CC16_414_5', 'CC16_414_6', 'CC16_417a_1', 'CC16_417a_2', 'CC16_417a_3', 'CC16_417a_4', 'CC16_417a_5', 'CC16_422d', 'CC16_422c', 'edloan', 'CC16_422f', 'CC16_422e', 'CC16_426_1', 'CC16_426_2', 'CC16_426_3', 'CC16_426_5', 'CC16_418a', 'CC16_426_4', 'CC16_401', 'CC16_326', 'CC16_410a']


In [21]:
df = df[variables]

I am interested in people who took the post-survey, indicating how they voted.

In [22]:
df = df[df['tookpost'] == 'Yes']

#df.info()

In [23]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse=True)

for i in range(1, len(nan_counts)):
    print(nan_counts[i])

('CC16_410a', 7657)
('hispanic', 3903)
('CC16_401', 3814)
('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('CC16_351H', 232)
('CC16_426_4', 207)
('CC16_351G', 188)
('CC16_330a', 186)
('CC16_418a', 176)
('CC16_426_5', 172)
('CC16_351F', 168)
('CC16_426_3', 167)
('CC16_426_2', 145)
('CC16_426_1', 139)
('immstat', 127)
('CC16_422e', 124)
('CC16_422f', 121)
('investor', 119)
('CC16_351E', 118)
('CC16_351B', 111)
('edloan', 110)
('CC16_327', 108)
('child18', 96)
('union', 87)
('CC16_351I', 82)
('CC16_332b', 78)
('CC16_332f', 75)
('CC16_332c', 66)
('CC16_422c', 65)
('trans', 62)
('CC16_422d', 62)
('ownhome', 61)
('CC16_333c', 55)
('CC16_332a', 53)
('CC16_332e', 53)
('CC16_333a', 52)
('CC16_334c', 49)
('CC16_333b', 48)
('CC16_332d', 48)
('CC16_333d', 44)
('CC16_334d', 39)
('sexuality', 31)
('CC16_334a', 28)
('CC16_351K', 16)
('faminc', 12)
('tookpost', 0)
('inputstate', 0)
('birthyr', 0)
('gender', 0)
('educ', 0)
('race', 0)
('CC16_331_1', 0)
('CC16

CC16_326 (2012 pres vote) has the most missing data when just considering post survey. Given how many there are, I will just exclude rather than try to impute it.

In [24]:
df = df.drop(['CC16_326'], axis = 1)

The variable with the next-largest number of missing rows is CC16_410a (Who did you vote for in 2016?). This is an essential outcome variable so I cannot exclude it. Fortunately, the missing rows come from respondents who did not answer this question because they already indicated that they did not vote for question CC16_401.

In [25]:
df.groupby(['CC16_401']).size()

CC16_401
I attempted to vote but did not or could not.      487
I definitely voted in the General Election.      45292
I did not vote in the election this November.     1802
I thought about voting this time - but didn't      750
I usually vote, but didn't this time.              754
dtype: int64

In [26]:
def voted(s):
    if "definitely voted" in str(s):
        return 1
    else:
        return 0

df['voted'] = df['CC16_401'].apply(voted)

In [27]:
df.groupby(['voted']).size()

voted
0     7607
1    45292
dtype: int64

In [28]:
df.groupby(['CC16_410a']).size()

CC16_410a
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election       81
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

In [29]:
df.loc[df['voted'] == 0, 'CC16_410a'] = 'I didn\'t vote in this election'

In [30]:
df.groupby(['CC16_410a']).size()

CC16_410a
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election     7688
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

In [32]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse=True)

for i in range(0, 10):
    print(nan_counts[i])

('hispanic', 3903)
('CC16_401', 3814)
('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('CC16_351H', 232)
('CC16_426_4', 207)
('CC16_351G', 188)


CC16_410a (presidential vote) is now almost entirely filled out. CC16_401 itself has a lot of missing data due to respondents not being asked that question, but the information is contained in CC16_410a anyway, so I won't worry about it.

In [33]:
df = df.drop(['CC16_401'], axis = 1)

The next column is the hispanic column. Again this comes from respondents not being asked the question, and I can get this data from the race column.

In [34]:
df.groupby(['race']).size()

race
Asian               1739
Black               4931
Hispanic            3742
Middle Eastern        99
Mixed               1169
Native American      440
Other                680
White              40099
dtype: int64

In [35]:
df.groupby(['hispanic']).size()

hispanic
No     47389
Yes     1607
dtype: int64

In [36]:
#fill in values of hispanic column based on race column - survey didn't ask hispanic question if they already selected hispanic for race
df.loc[df['race'] == 'Hispanic', 'hispanic'] = 'Yes'

In [37]:
df.groupby(['hispanic']).size()

hispanic
No     47389
Yes     5349
dtype: int64

In [38]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse = True)

for i in range(0, 10):
    print(nan_counts[i])

('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('CC16_351H', 232)
('CC16_426_4', 207)
('CC16_351G', 188)
('CC16_330a', 186)
('CC16_418a', 176)


For the other columns, missing values come from respondents skipping the question.

In [39]:
cleaned_df = df.dropna(axis = 0, how = 'any')
len(cleaned_df.index)

48592

Dropping all rows with missing values results in 48592 remaining, out of 52899 total respondents who responded to the post-survey. This isn't terrible but I can get that down by imputing values for some columns.

Next cleaning steps: turning categorical variables into dummies, which probably requires collapsing some into fewer categories first.

In [40]:
df.to_csv('cleaned.csv')